In [ ]:
import random
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
words = open("../names.txt").read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = dict([(c, i) for i, c in enumerate(['.']+chars)])
itos = dict([(i, c) for c, i in stoi.items()])
vocab_size = len(stoi.keys())
block_size = 8 # Context length

In [ ]:
# Making X and Y
def build_dataset(words):
    X, Y = [], []
    
    for w in words:
        context = [0] * block_size # Starting with 3 dots. Default that leads to the first Y token prediction.
        
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print("".join([itos[i] for i in context]) + " --> " + ch)
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

random.seed = 42
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

# Train, dev, test
Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

In [ ]:
Xtr.shape

In [ ]:
# Let's implement the pytorch layers to jog our memory.
class Linear:
    def __init__(self, fan_in, fan_out, bias=False):
        self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5
        self.bias = torch.randn(fan_out) if bias is True else None

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

class BatchNorm1d:
    def __init__(self, dim, momentum=0.1, eps=1e-5):
        self.training = True
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        self.momentum = momentum
        self.eps = eps

    def __call__(self, x):
        if self.training:
            # fixing the batch norm bug for the new unit batch for wavenet
            if x.ndim == 2:
                dim = 0
            elif x.ndim == 3:
                dim = (0, 1)
            xmean = x.mean(dim, keepdims=True)
            xvar = x.var(dim, keepdims=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        if self.training is True:
            with torch.no_grad():
                self.running_mean = self.running_mean * (1 - self.momentum) + self.momentum * xmean
                self.running_var = self.running_var * (1 - self.momentum) + self.momentum * xvar
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out

    def parameters(self):
        return []

class Embedding:
    def __init__(self, num_embeddings, embed_dim):
        self.weight = torch.randn((num_embeddings, embed_dim))

    def __call__(self, ix):
        self.out = self.weight[ix]
        return self.out

    def parameters(self):
        return [self.weight]

class Flatten:
    def __call__(self, x):
        self.out = x.view(x.shape[0], -1)
        return self.out

    def parameters(self):
        return []

# Implementing Wavenet
class FlattenConsecutive:
    def __init__(self, n):
        self.n = n
        
    def __call__(self, x):
        B, T, C = x.shape
        x = x.view(B, T//self.n, C*self.n)
        if x.shape[1] == 1:
            x = x.squeeze(1)
        self.out = x
        return self.out

    def parameters(self):
        return []

class Sequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, x):
        self.out = x
        for layer in self.layers:
            x = layer(x)
        self.out = x
        return self.out

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

In [ ]:
torch.manual_seed(42)

In [ ]:
ix = torch.randint(0, Xtr.shape[0], (4,))
Xs, Ys = Xtr[ix], Ytr[ix]
logits = model(Xs)
loss = F.cross_entropy(logits, Ys)

In [ ]:
for layer in model.layers:
    print(layer.__class__.__name__, layer.out.shape)

In [ ]:
model.layers[-7].running_var.shape

In [ ]:
n_emb = 24
#n_hidden = 200
#n_hidden = 68
n_hidden = 128
unit = 2

# Wavenet style
model = Sequential([
    Embedding(vocab_size, n_emb),
    FlattenConsecutive(unit), Linear(n_emb * unit, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    FlattenConsecutive(unit), Linear(n_hidden * unit, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    FlattenConsecutive(unit), Linear(n_hidden * unit, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, vocab_size),
])

with torch.no_grad():
    # last layer: make less confident
    #layers[-1].gamma *= 0.1
    model.layers[-1].weight *= 0.1
    # all other layers: apply gain
    for layer in model.layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 5/3

parameters = [p for l in model.layers for p in l.parameters()]

for p in parameters:
    p.requires_grad = True

stepi = []
lossi = []
ud = []
batch = 32

In [ ]:
epochs = 200000
lr = 0.01

for epoch in range(epochs):
    ix = torch.randint(0, Xtr.shape[0], (batch,))
    Xb, Yb = Xtr[ix], Ytr[ix]
    logits = model(Xb)
    loss = F.cross_entropy(logits, Yb)
    for layer in model.layers:
        layer.out.retain_grad()
    for p in parameters:
        p.grad = None
    loss.backward()

    lr = 0.1 if epoch < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    stepi.append(epoch)
    lossi.append(loss.log10().item())
    if epoch % 10000 == 0:
        print(f'{epoch:7d}/{epochs:7d}: {loss.item():.4f}')
    
    with torch.no_grad():
        ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])

    # if epoch == 1000:
    #     break

In [ ]:
plt.plot(torch.tensor(lossi).view(-1, 1000).mean(1));

In [ ]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
    x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xtest, Ytest),
    }[split]
    logits = model(x)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

# put layers into eval mode
for layer in model.layers:
  layer.training = False
split_loss('train')
split_loss('val')

### MLP + BatchNorm1d
_train_: 1.7142295837402344
_val_: 1.9895931482315063
### WaveNet - emb: 10, nhidden: 64
_train_: 1.9279804229736328
_val_: 2.041594982147217
### WaveNet - emb: 24, nhidden: 128
_train_: 1.793333649635315
_val_: 2.0019969940185547